# 0. Setup
We will begin by importing all required libraries and fixing the plotting setup.

In [1]:
#imports
import pandas as pd
import numpy as np
import pickle
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import matplotlib.colors as mcolors
import seaborn as sns
import random
import os
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
from sklearn.cluster import Birch
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn import mixture
from sklearn.mixture import BayesianGaussianMixture
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import distance
#colab specific: ######################################################################
!pip install umap-learn[plot]
!pip install holoviews
!pip install -U ipykernel
#######################################################################################
import umap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55622 sha256=9f812a0a2671a99fba6619319ab753b01e37207f2d385168875cc1235e36661b
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82816 sha256=1dd2370ccdb75587598e0ee6a6007a3ea08a7e57486bcd9bf2f4bc8d9001bfba
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for dat

In [2]:
#plot settings
%matplotlib inline
sns.set(style='white', context='notebook', rc={'figure.figsize':(3,3)})
plt.ioff() #ensures that no plots are shown within the notebook if not explicity demanded by plt.show()

#color configuration
colors = list(mcolors.CSS4_COLORS.keys())
colors = random.sample(colors, 100)
colors = np.array(colors)

# 1. Setting the Parameters
For a better legibility of this document, we will store all parameter settings in this section.

In [3]:
#Path to workspace
w_path = '/content/gdrive/MyDrive/DIm_red_Clustering/'

#Pre-processing of the imported data - choose between...
# 'feature_stand': feature standardization leading to unit vairance and zero mean of all features across the samples
# 'norm_vecs': Normalized embedding vectors, that project all embedding vectors on a unit sphere
# 'none'
pre_processing = 'norm_vecs'


#Dimensionality reduction of the imported data - choose between...
# 'PCA'
# 'UMAP'
dim_reduction = 'UMAP'


#Data-generation network - choose between...
# 'Mannheim':
# 'adapters':
data_generation = 'final'

# 2. Loading the embeddings

Now, we will have to load the user embeddings from the npy file they are stored in.

We also want to check that the imported data has the desired dimensions, to make sure that nothing went wrong throughout the process of creating and storing the embeddings in the npy file, and importing them into this document.

In [ ]:
#colab specific import setup: ######################################################################
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
####################################################################################################

#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_final.npy'

#loading the data
data = np.load(path)
print(path)

#create folder to sort images into
data_gen_path = w_path + data_generation
if os.path.exists(data_gen_path) == False:
  os.mkdir(data_gen_path)

#Check that embeddings have the correct shape
print(data.shape)

Mounted at /content/gdrive


# 3. Pre-processing the embeddings
Before passing our user embeddings on to the dimensionality reduction, we will have to pre-process them, to make sure that not a few features only dominate the dimensionality reduction due to scale differences.

In [ ]:
#checking which means of data pre-processing to use
if pre_processing == 'norm_vecs':
  scale_factors = np.sum(data, axis = 1)
  scale_factors = scale_factors[:, np.newaxis]
  processed_data = data / scale_factors
elif pre_processing =='feature_stand':
  processed_data = StandardScaler().fit_transform(data)
else:
  processed_data = data

#create folder to sort images into
pre_processing_path = data_gen_path + '/' + pre_processing
if os.path.exists(pre_processing_path) == False:
  os.mkdir(pre_processing_path)


# 4. Performing Dimensionality Reduction


In [ ]:
#initializing dict to save reduced dimensionality embeddings in, and to retrieve those embeddings based on the parameters used to create them
embeddings = {}


In [ ]:
#Create folders to later deposit images in
path_dimred = pre_processing_path + '/' + dim_reduction
os.mkdir(path_dimred)
path2D = path_dimred + '/' + str(2)
os.mkdir(path2D)
path3D = path_dimred + '/' + str(3)
os.mkdir(path3D)

#print(path2D)
print(path3D)

#Checking which means of dimensionality reduction to use
if dim_reduction == 'UMAP':

  #UMAP
  #iterating over plausible hyperparameter values: Dimension, number of neighbors, minimum distance and distance metric
  for n_dims in [3]:
    for n_neighbors in np.concatenate((range(2,11,1), range(15,51,5))): #n_neighbors in steps of 1 from 2-10
      for min_dist in range(0,10,1): #min_dist between 0 and 1 in steps of 0.1
        for measure in ['euclidean','manhattan','cosine']: #different distance measures

          reducer = umap.UMAP(n_components = n_dims, n_neighbors = n_neighbors, min_dist = min_dist/10, metric = measure) #initialize umap with desired hyperparams
          reduced_data = reducer.fit_transform(processed_data)#calculate umap and return reduced embeddings
          embeddings[data_generation+ pre_processing + dim_reduction + str(n_dims) + str(n_neighbors)+ str(min_dist/10) + measure] = reduced_data #saving embeddings for retrieval during clustering

          if n_dims == 2:
            #plotting result and saving plots as images
            plt.figure()
            plt.scatter(reduced_data[:, 0], reduced_data[:, 1], s=0.1)
            plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + str(n_neighbors)+ '     min_dist:' + str(min_dist/10) + '     metric:' + measure, fontsize=8)
            path2D_image = path2D + '/' + str(n_neighbors) + '_' + str(min_dist/10) + '_' + measure +'.pdf'
            plt.savefig(path2D_image, pad_inches = 15)

          else:
            #plotting result and saving plots as images
            plt.figure()
            ax = plt.axes(projection='3d')
            ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2],s=0.1)
            plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + str(n_neighbors)+ '     min_dist:' + str(min_dist/10) + '     metric:' + measure, fontsize=8)
            path3D_image = path3D + '/' + str(n_neighbors) + '_' + str(min_dist/10) + '_' + measure +'.pdf'
            plt.savefig(path3D_image, pad_inches = 15)

    if os.path.exists(data_gen_path +'/' + 'reduced_embeddings') == False:
      os.mkdir(data_gen_path +'/' +  'reduced_embeddings')

    with open(data_gen_path + '/' + 'reduced_embeddings'+ '/' + 'red_embeddings_UMAP_' + pre_processing + str(n_dims) + '.pickle', 'wb') as file:
      pickle.dump(embeddings, file)


else:
  #PCA
  #iterating over hyperparameter: Dimension
  for n_dims in [2,3]:
    pca = PCA(n_components = n_dims)
    reduced_data = pca.fit_transform(processed_data)
    embeddings[data_generation+ pre_processing + dim_reduction + str(n_dims) + 'na'+ 'na' + 'na'] = reduced_data #saving embeddings for retrieval during clustering

    if n_dims == 2:
      #plotting result and saving plots as images
      plt.figure()
      plt.scatter(reduced_data[:, 0], reduced_data[:, 1], s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction, fontsize=8)
      path2D_image = path2D + '/'+ 'PCA' + '.pdf'
      plt.savefig(path2D_image, pad_inches = 15)

    else:
      #plotting result and saving plots as images
      plt.figure()
      ax = plt.axes(projection='3d')
      ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2],s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction, fontsize=8)
      path3D_image = path3D + '/' + 'PCA' +'.pdf'
      plt.savefig(path3D_image, pad_inches = 15)

    with open(data_gen_path + '/' + 'reduced_embeddings'+ '/' + 'red_embeddings_PCA' + pre_processing + '.pickle', 'wb') as file:
      pickle.dump(embeddings, file)



/content/gdrive/MyDrive/DIm_red_Clustering/final/norm_vecs/UMAP/3


/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ma

# 5. Selecting suitable reduced embeddings
As a next step, we will have to visually inspect the resulting plots to determine the most suitable reduced dimensionality embeddings. We will note down the hyperparamters used to create these embeddings.

The chosen embeddings can be retrieved later when clustering by listing the hyperparameters used to create them as a keyword for the "*embeddings*" dictionary.

In [ ]:
#extracting the saved embeddings from our files

umap_feature_stand2 = {}
umap_feature_stand3 = {}
umap_norm_vecs2 = {}
umap_norm_vecs3 = {}
umap_none2 = {}
umap_none3 = {}
pca = {}

with open(w_path + data_generation + '/' + 'reduced_embeddings'+ '/' + 'red_embeddings_UMAP_none2.pickle', 'rb') as file:
    umap_none2 = pickle.load(file)

with open(w_path + data_generation + '/' + 'reduced_embeddings'+ '/' + 'red_embeddings_UMAP_none3.pickle', 'rb') as file:
    umap_none3 = pickle.load(file)

with open(w_path + data_generation + '/' + 'reduced_embeddings'+ '/' + 'red_embeddings_UMAP_feature_stand2.pickle', 'rb') as file:
    umap_feature_stand2 = pickle.load(file)

with open(w_path + data_generation + '/' + 'reduced_embeddings'+ '/' + 'red_embeddings_UMAP_feature_stand3.pickle', 'rb') as file:
    umap_feature_stand3 = pickle.load(file)

with open(w_path + data_generation + '/' + 'reduced_embeddings'+ '/' + 'red_embeddings_UMAP_norm_vecs2.pickle', 'rb') as file:
    umap_norm_vex2 = pickle.load(file)

with open(w_path + data_generation + '/' + 'reduced_embeddings'+ '/' + 'red_embeddings_UMAP_norm_vecs3.pickle', 'rb') as file:
    umap_norm_vex3 = pickle.load(file)

with open(w_path + data_generation + '/' + 'reduced_embeddings'+ '/' + 'red_embeddings_PCA.pickle', 'rb') as file:
    pca = pickle.load(file)

#Merging the saved embeddings into one dictionary
embeddings = umap_none2 | umap_none3 | umap_feature_stand2 | umap_feature_stand3 | umap_norm_vex2 | umap_norm_vex3 | pca


In [ ]:
#selecting best values from previous dimensionality reduction and placing them in iterable array

data_gen_array = np.full((34,), 'final')

pre_proc_array_1 = np.full((21,), 'feature_stand')
pre_proc_array_2 = np.full((13,), 'norm_vecs')
pre_proc_array = np.concatenate((pre_proc_array_1, pre_proc_array_2))

dim_red_array = np.full((34,),'UMAP')

n_dims_array_1 = np.full((10,),'2')
n_dims_array_2 = np.full((11,), '3')
n_dims_array_3 = np.full((7,),'2')
n_dims_array_4 = np.full((6,), '3')
n_dims_array = np.concatenate((n_dims_array_1, n_dims_array_2, n_dims_array_3, n_dims_array_4))

neighbors_array = np.full((34,), '2')

dist_array = ['0.4', '0.3', '0.6', '0.3', '0.7', '0.9', '0.3', '0.7', '0.5', '0.4', '0.4', '0.3', '0.6', '0.5', '0.6', '0.5', '0.6', '0.7','0.7','0.8','0.5','0.4','0.4','0.6','0.8','0.9','0.5','0.7','0.1', '0.2', '0.5', '0.6', '0.4','0.3']

metric_array = ['cosine','cosine','cosine','euclidean','euclidean','euclidean','euclidean','manhattan','euclidean','euclidean','cosine','cosine','cosine','cosine','euclidean','manhattan','manhattan','euclidean','manhattan','euclidean','euclidean','euclidean','manhattan','euclidean','euclidean','euclidean','manhattan','euclidean','euclidean','manhattan','manhattan','manhattan','manhattan','euclidean']

iter_array = np.column_stack((data_gen_array, pre_proc_array, dim_red_array, n_dims_array, neighbors_array, dist_array, metric_array))


#6. Checking Dimensionality Reduction Validity
We will proceed by validating the chosen means of dimensionality reduction.

##6.1 Sanity Check: Transforming Test Data to Lower Dimension
We now want to investigate whether the learned dimensionality reduction performed on a train set of embeddings also projects the test set of embeddings to the existing clusters.

In [ ]:
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


#loading the data
data = np.load(path)

if os.path.exists(w_path +'/' + 'UMAP_testing2') == False:
  os.mkdir(w_path +'/' +  'UMAP_testing')

for i in range(5):
  for test_size in [0.1, 0.05]:
    for pre_processing in ['feature_stand','norm_vecs']:

      train_data, test_data = train_test_split(data, test_size= test_size)

      #checking which means of data pre-processing to use
      if pre_processing == 'norm_vecs':
        scale_factors_train = np.sum(train_data, axis = 1)
        scale_factors_train = scale_factors_train[:, np.newaxis]
        scaled_train_data = train_data / scale_factors_train
        scale_factors_test = np.sum(test_data, axis = 1)
        scale_factors_test = scale_factors_test[:, np.newaxis]
        scaled_test_data = test_data / scale_factors_test
      else:
        mean = np.mean(train_data, axis = 0)
        mean = mean[:, np.newaxis]
        std = np.std(train_data, axis = 0)
        std = std[:, np.newaxis]
        scaled_train_data = ((train_data.T - mean)/std).T
        scaled_test_data = ((test_data.T - mean)/std).T
        mean = np.mean(scaled_train_data, axis = 0)
        var = np.var(scaled_train_data, axis = 0)


      umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'euclidean') #initialize umap with desired hyperparams
      reduced_train_data = umap_reducer.fit_transform(scaled_train_data) #calculate umap and return reduced embeddings
      reduced_test_data = umap_reducer.transform(scaled_test_data) #calculate umap and return reduced embeddings
      plt.figure()
      plt.title(pre_processing + '   test_set_size:' + str(test_size*100) +'%' + '   nr:' + str(i+1) )
      plt.scatter(reduced_train_data[:, 0], reduced_train_data[:, 1], s=0.1, color = 'silver')
      plt.scatter(reduced_test_data[:, 0], reduced_test_data[:, 1], s=0.1, color = 'crimson')
      plt.savefig(w_path + '/' + 'UMAP_testing' + '/' + pre_processing + 'test_set_size' + str(test_size*100) +'%' + '_nr' + str(i+1) + '.pdf', bbox_inches='tight')



While the results look quite nice in the case of vector normalization, they do not look appropriate when using feature_standardization.

Our first thought was that maybe the test data influence the feature standardization's mean and std parameters more than previously assumed. However, when looking at the difference between the feature means and stds of the entire embeddings data set and the feature means and stds of just the train embeddings, the difference is marginal and hence negligible. This can thus not be the source of our problem.

In [ ]:
#Checking if sampling has a relevant impact on feature standardization
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


#loading the data
data = np.load(path)

train_data, test_data = train_test_split(data, test_size= 0.1)

for i in range(5):
#checking which means of data pre-processing to use
  mean = np.mean(train_data, axis = 0)
  #mean = mean[:, np.newaxis]
  std = np.std(train_data, axis = 0)
  #std = std[:, np.newaxis]

  total_mean = np.mean(data, axis = 0)
  total_std = np.std(data,axis = 0)

  d_mean = total_mean - mean
  d_std = total_std - std

  print(d_mean)
  print('\n')
  print(d_std)


Next we will see that UMAP is not deterministic. If we perform UMAP twice with the same hyperparams and on the same data, the resulting plots will look slightly different.

In [ ]:
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


#loading the data
data = np.load(path)

if os.path.exists(w_path +'/' + 'UMAP_twoiterations') == False:
  os.mkdir(w_path +'/' +  'UMAP_twoiterations')

for i in range(5):
  for test_size in [0.1, 0.01]:
    for pre_processing in ['feature_stand','norm_vecs']:

      train_data, test_data = train_test_split(data, test_size= test_size)

      #checking which means of data pre-processing to use
      if pre_processing == 'norm_vecs':
        scale_factors_train = np.sum(train_data, axis = 1)
        scale_factors_train = scale_factors_train[:, np.newaxis]
        scaled_train_data = train_data / scale_factors_train
        scale_factors_test = np.sum(test_data, axis = 1)
        scale_factors_test = scale_factors_test[:, np.newaxis]
        scaled_test_data = test_data / scale_factors_test

        scale_factors = np.sum(data, axis = 1)
        scale_factors = scale_factors[:, np.newaxis]
        scaled_data = data / scale_factors
      else:

        mean = np.mean(train_data, axis = 0)
        mean = mean[:, np.newaxis]
        std = np.std(train_data, axis = 0)
        std = std[:, np.newaxis]
        scaled_train_data = ((train_data.T - mean)/std).T
        scaled_test_data = ((test_data.T - mean)/std).T
        mean = np.mean(scaled_train_data, axis = 0)
        var = np.var(scaled_train_data, axis = 0)

        total_mean = np.mean(data, axis = 0)
        total_std = np.std(data,axis = 0)
        total_std = total_std[:, np.newaxis]
        total_mean = total_mean[:, np.newaxis]
        scaled_data = scaled_data = ((data.T - total_mean)/total_std).T


      umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'euclidean') #initialize umap with desired hyperparams
      reduced_data1 = umap_reducer.fit_transform(scaled_data) #calculate umap and return reduced embeddings
      #reduced_test_data = umap_reducer.transform(scaled_test_data) #calculate umap and return reduced embeddings
      umap_total_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'euclidean') #initialize umap with desired hyperparams
      reduced_data2 = umap_total_reducer.fit_transform(scaled_data)
      plt.figure()
      plt.title(pre_processing + '   test_set_size:' + str(test_size*100) +'%' + '   nr:' + str(i+1) )
      plt.scatter(reduced_data1[:, 0], reduced_data1[:, 1], s=0.1, color = 'silver')
      #plt.scatter(reduced_test_data[:, 0], reduced_test_data[:, 1], s=0.1, color = 'crimson')
      plt.scatter(reduced_data2[:, 0], reduced_data2[:, 1], s=0.1, color = 'olive')
      plt.savefig(w_path + '/' + 'UMAP_twoiterations' + '/' + pre_processing + 'test_set_size' + str(test_size*100) +'%' + '_nr' + str(i+1) + '.2' + '.pdf', bbox_inches='tight')


We want to see if maybe we can solve this issue by averaging over several UMAP results. This however doesn't always work, as can be seen in the resulting plots, and is thus not a reliable solution.

In [ ]:
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


if os.path.exists(w_path +'/' + 'UMAP_averaging') == False:
  os.mkdir(w_path +'/' +  'UMAP_averaging')
#loading the data
data = np.load(path)
train_data, test_data = train_test_split(data, test_size= test_size)

embeddings_total = []
embeddings_train = []

for i in range(10):
  for test_size in [0.1]:
    for pre_processing in ['feature_stand']:



      #checking which means of data pre-processing to use
      if pre_processing == 'norm_vecs':
        scale_factors_train = np.sum(train_data, axis = 1)
        scale_factors_train = scale_factors_train[:, np.newaxis]
        scaled_train_data = train_data / scale_factors_train
        scale_factors_test = np.sum(test_data, axis = 1)
        scale_factors_test = scale_factors_test[:, np.newaxis]
        scaled_test_data = test_data / scale_factors_test

        scale_factors = np.sum(data, axis = 1)
        scale_factors = scale_factors[:, np.newaxis]
        scaled_data = data / scale_factors
      else:
        #sc = StandardScaler()
        #sc.fit(train_data)
        #scaled_train_data = sc.transform(train_data)
        #scaled_test_data = sc.transform(test_data)
        #X_orig= sc.inverse_tranform(X_scaled)
        mean = np.mean(train_data, axis = 0)
        mean = mean[:, np.newaxis]
        #mean_train = np.broadcast_to(mean, (mean.shape[0], train_data.shape[0]))
        #print(mean_train.shape)
        #mean_test = np.broadcast_to(mean, (mean.shape[0], test_data.shape[0]))
        std = np.std(train_data, axis = 0)
        std = std[:, np.newaxis]
        scaled_train_data = ((train_data.T - mean)/std).T
        scaled_test_data = ((test_data.T - mean)/std).T
        mean = np.mean(scaled_train_data, axis = 0)
        var = np.var(scaled_train_data, axis = 0)

        total_mean = np.mean(data, axis = 0)
        total_std = np.std(data,axis = 0)
        total_std = total_std[:, np.newaxis]
        total_mean = total_mean[:, np.newaxis]
        scaled_data = scaled_data = ((data.T - total_mean)/total_std).T


      umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'euclidean') #initialize umap with desired hyperparams
      reduced_train_data = umap_reducer.fit_transform(scaled_train_data) #calculate umap and return reduced embeddings
      reduced_test_data = umap_reducer.transform(scaled_test_data) #calculate umap and return reduced embeddings
      umap_total_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'euclidean') #initialize umap with desired hyperparams
      reduced_data = umap_total_reducer.fit_transform(scaled_data)
      embeddings_total.append(reduced_data)
      embeddings_train.append(reduced_train_data)



#embeddings_total_avg = np.mean( np.array([ embeddings_total[0], embeddings_total[1], embeddings_total[2], embeddings_total[3],embeddings_total[4],embeddings_total[5],embeddings_total[6],embeddings_total[7],embeddings_total[8],embeddings_total[9],embeddings_total[10], embeddings_total[11], embeddings_total[12], embeddings_total[13],embeddings_total[14],embeddings_total[15],embeddings_total[16],embeddings_total[17],embeddings_total[18],embeddings_total[19] ]), axis=0 )
#embeddings_train_avg = np.mean( np.array([ embeddings_train[0], embeddings_train[1], embeddings_train[2], embeddings_train[3],embeddings_train[4],embeddings_train[5],embeddings_train[6],embeddings_train[7],embeddings_train[8],embeddings_train[9],embeddings_train[10], embeddings_train[11], embeddings_train[12], embeddings_train[13],embeddings_train[14],embeddings_train[15],embeddings_train[16],embeddings_train[17],embeddings_train[18],embeddings_train[19] ]), axis=0 )#
#embeddings_total_avg = np.mean( np.array([ embeddings_total[0], embeddings_total[1], embeddings_total[2], embeddings_total[3],embeddings_total[4] ]), axis=0 )
#embeddings_train_avg = np.mean( np.array([ embeddings_train[0], embeddings_train[1], embeddings_train[2], embeddings_train[3],embeddings_train[4]]), axis=0 )
embeddings_total_avg = np.mean( np.array([ embeddings_total[0], embeddings_total[1], embeddings_total[2], embeddings_total[3],embeddings_total[4],embeddings_total[5],embeddings_total[6],embeddings_total[7],embeddings_total[8],embeddings_total[9] ]), axis=0 )
embeddings_train_avg = np.mean( np.array([ embeddings_train[0], embeddings_train[1], embeddings_train[2], embeddings_train[3],embeddings_train[4],embeddings_train[5],embeddings_train[6],embeddings_train[7],embeddings_train[8],embeddings_train[9] ]), axis=0 )
plt.figure()
plt.title(pre_processing + '   test_set_size:' + str(test_size*100) +'%' + '   nr:' + str(i+1) )
plt.scatter(embeddings_train_avg[:, 0], embeddings_train_avg[:, 1], s=0.1, color = 'silver')
#plt.scatter(reduced_test_data[:, 0], reduced_test_data[:, 1], s=0.1, color = 'crimson')
plt.scatter(embeddings_total_avg[:, 0], embeddings_total_avg[:, 1], s=0.1, color = 'olive')
plt.savefig(w_path + '/' + 'UMAP_averaging' + '/' + pre_processing + 'test_set_size' + str(test_size*100) +'%' + '_nr' + str(i+1) + '.2' + '.pdf', bbox_inches='tight')

We'll now see how this works without doing any data pre-processing at all.

In [ ]:
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


#loading the data
data = np.load(path)

if os.path.exists(w_path +'/' + 'UMAP_transform_wo_pp') == False:
  os.mkdir(w_path +'/' +  'UMAP_transform_wo_pp')

for i in range(5):
  for test_size in [0.1, 0.05]:

    train_data, test_data = train_test_split(data, test_size= test_size)



    umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'euclidean') #initialize umap with desired hyperparams
    reduced_train_data = umap_reducer.fit_transform(train_data) #calculate umap and return reduced embeddings
    reduced_test_data = umap_reducer.transform(test_data) #calculate umap and return reduced embeddings
    plt.figure()
    plt.title(pre_processing + '   test_set_size:' + str(test_size*100) +'%' + '   nr:' + str(i+1) )
    plt.scatter(reduced_train_data[:, 0], reduced_train_data[:, 1], s=0.1, color = 'silver')
    plt.scatter(reduced_test_data[:, 0], reduced_test_data[:, 1], s=0.1, color = 'crimson')
    plt.savefig(w_path + '/' + 'UMAP_transform_wo_pp' + '/' + 'test_set_size' + str(int(test_size*100)) +'%' + '_nr' + str(i+1) + '.pdf', bbox_inches='tight')



Finally, we will try a different approach. We will determine the k nearest neighbors of the test data embedding and interpolate between their lower-dim embeddings to calculate the test data embedding.

In [ ]:
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


#loading the data
data = np.load(path)

if os.path.exists(w_path +'/' + 'UMAP_cosine_interpolation') == False:
  os.mkdir(w_path +'/' +  'UMAP_cosine_interpolation')

for i in range(5):
  for test_size in [0.1, 0.05]:
    for pre_processing in ['feature_stand']:

      train_data, test_data = train_test_split(data, test_size= test_size)
      nn = NearestNeighbors(n_neighbors=4)
      neighbors = np.empty((test_data.shape[0],3))
      for a in range(test_data.shape[0]):
        test_data_item = test_data[a]
        test_data_item = test_data_item[np.newaxis,:]
        #train_data_transposed = train_data

        total_data = np.concatenate((train_data, test_data_item), axis = 0)

        nn.fit(total_data)
        _ , neighbor = nn.kneighbors(test_data_item)


        neighbors[a][:] = neighbor[0][1:4]




      #checking which means of data pre-processing to use
      if pre_processing == 'norm_vecs':
        scale_factors_train = np.sum(train_data, axis = 1)
        scale_factors_train = scale_factors_train[:, np.newaxis]
        scaled_train_data = train_data / scale_factors_train
        scale_factors_test = np.sum(test_data, axis = 1)
        scale_factors_test = scale_factors_test[:, np.newaxis]
        scaled_test_data = test_data / scale_factors_test
      else:
        #sc = StandardScaler()
        #sc.fit(train_data)
        #scaled_train_data = sc.transform(train_data)
        #scaled_test_data = sc.transform(test_data)
        #X_orig= sc.inverse_tranform(X_scaled)
        mean = np.mean(train_data, axis = 0)
        mean = mean[:, np.newaxis]
        #mean_train = np.broadcast_to(mean, (mean.shape[0], train_data.shape[0]))
        #print(mean_train.shape)
        #mean_test = np.broadcast_to(mean, (mean.shape[0], test_data.shape[0]))
        std = np.std(train_data, axis = 0)
        std = std[:, np.newaxis]
        scaled_train_data = ((train_data.T - mean)/std).T
        scaled_test_data = ((test_data.T - mean)/std).T
        mean = np.mean(scaled_train_data, axis = 0)
        var = np.var(scaled_train_data, axis = 0)

      print(test_data.shape)
      umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'cosine') #initialize umap with desired hyperparams
      reduced_train_data = umap_reducer.fit_transform(scaled_train_data) #calculate umap and return reduced embeddings
      reduced_test_data = np.empty((test_data.shape[0], reduced_train_data.shape[1]))
      for j in range(test_data.shape[0]):
        #print(neighbors[j][0])
        #print(reduced_train_data[int(neighbors[j][0])])
        reduced_test_data[j] = np.mean( np.array([reduced_train_data[int(neighbors[j][0])], reduced_train_data[int(neighbors[j][1])], reduced_train_data[int(neighbors[j][2])]]), axis = 0)

      plt.figure()
      plt.title(pre_processing + '   test_set_size:' + str(test_size*100) +'%' + '   nr:' + str(i+1) )
      plt.scatter(reduced_train_data[:, 0], reduced_train_data[:, 1], s=0.1, color = 'silver')
      plt.scatter(reduced_test_data[:, 0], reduced_test_data[:, 1], s=0.1, color = 'crimson')
      plt.savefig(w_path + '/' + 'UMAP_cosine_interpolation' + '/' + pre_processing + 'test_set_size' + str(test_size*100) +'%' + '_nr' + str(i+1) + '.pdf', bbox_inches='tight')



In [ ]:
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


#loading the data
data = np.load(path)

if os.path.exists(w_path +'/' + 'UMAP_interpolation') == False:
  os.mkdir(w_path +'/' +  'UMAP_interpolation')

for i in range(5):
  for test_size in [0.1, 0.05]:
    for pre_processing in ['feature_stand','norm_vecs']:

      train_data, test_data = train_test_split(data, test_size= test_size)
      nn = NearestNeighbors(n_neighbors=4)
      neighbors = np.empty((test_data.shape[0],3))
      for a in range(test_data.shape[0]):
        test_data_item = test_data[a]
        test_data_item = test_data_item[np.newaxis,:]
        #train_data_transposed = train_data

        total_data = np.concatenate((train_data, test_data_item), axis = 0)

        nn.fit(total_data)
        _ , neighbor = nn.kneighbors(test_data_item)


        neighbors[a][:] = neighbor[0][1:4]




      #checking which means of data pre-processing to use
      if pre_processing == 'norm_vecs':
        scale_factors_train = np.sum(train_data, axis = 1)
        scale_factors_train = scale_factors_train[:, np.newaxis]
        scaled_train_data = train_data / scale_factors_train
        scale_factors_test = np.sum(test_data, axis = 1)
        scale_factors_test = scale_factors_test[:, np.newaxis]
        scaled_test_data = test_data / scale_factors_test
      else:
        #sc = StandardScaler()
        #sc.fit(train_data)
        #scaled_train_data = sc.transform(train_data)
        #scaled_test_data = sc.transform(test_data)
        #X_orig= sc.inverse_tranform(X_scaled)
        mean = np.mean(train_data, axis = 0)
        mean = mean[:, np.newaxis]
        #mean_train = np.broadcast_to(mean, (mean.shape[0], train_data.shape[0]))
        #print(mean_train.shape)
        #mean_test = np.broadcast_to(mean, (mean.shape[0], test_data.shape[0]))
        std = np.std(train_data, axis = 0)
        std = std[:, np.newaxis]
        scaled_train_data = ((train_data.T - mean)/std).T
        scaled_test_data = ((test_data.T - mean)/std).T
        mean = np.mean(scaled_train_data, axis = 0)
        var = np.var(scaled_train_data, axis = 0)

      print(test_data.shape)
      umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'euclidean') #initialize umap with desired hyperparams
      reduced_train_data = umap_reducer.fit_transform(scaled_train_data) #calculate umap and return reduced embeddings
      reduced_test_data = np.empty((test_data.shape[0], reduced_train_data.shape[1]))
      for j in range(test_data.shape[0]):
        #print(neighbors[j][0])
        #print(reduced_train_data[int(neighbors[j][0])])
        reduced_test_data[j] = np.mean( np.array([reduced_train_data[int(neighbors[j][0])], reduced_train_data[int(neighbors[j][1])], reduced_train_data[int(neighbors[j][2])]]), axis = 0)

      plt.figure()
      plt.title(pre_processing + '   test_set_size:' + str(test_size*100) +'%' + '   nr:' + str(i+1) )
      plt.scatter(reduced_train_data[:, 0], reduced_train_data[:, 1], s=0.1, color = 'silver')
      plt.scatter(reduced_test_data[:, 0], reduced_test_data[:, 1], s=0.1, color = 'crimson')
      plt.savefig(w_path + '/' + 'UMAP_interpolation' + '/' + pre_processing + 'test_set_size' + str(test_size*100) +'%' + '_nr' + str(i+1) + '.pdf', bbox_inches='tight')



In [ ]:
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


#loading the data
data = np.load(path)

if os.path.exists(w_path +'/' + 'UMAP_cosine_afterpreprocessing') == False:
  os.mkdir(w_path +'/' +  'UMAP_cosine_afterpreprocessing')

for i in range(5):
  for test_size in [0.3, 0.1, 0.05]:
    for pre_processing in ['feature_stand']:

      train_data, test_data = train_test_split(data, test_size= test_size)
      nn = NearestNeighbors(n_neighbors=4)
      neighbors = np.empty((test_data.shape[0],3))


      #checking which means of data pre-processing to use
      if pre_processing == 'norm_vecs':
        scale_factors_train = np.sum(train_data, axis = 1)
        scale_factors_train = scale_factors_train[:, np.newaxis]
        scaled_train_data = train_data / scale_factors_train
        scale_factors_test = np.sum(test_data, axis = 1)
        scale_factors_test = scale_factors_test[:, np.newaxis]
        scaled_test_data = test_data / scale_factors_test
      else:
        #sc = StandardScaler()
        #sc.fit(train_data)
        #scaled_train_data = sc.transform(train_data)
        #scaled_test_data = sc.transform(test_data)
        #X_orig= sc.inverse_tranform(X_scaled)
        mean = np.mean(train_data, axis = 0)
        mean = mean[:, np.newaxis]
        #mean_train = np.broadcast_to(mean, (mean.shape[0], train_data.shape[0]))
        #print(mean_train.shape)
        #mean_test = np.broadcast_to(mean, (mean.shape[0], test_data.shape[0]))
        std = np.std(train_data, axis = 0)
        std = std[:, np.newaxis]
        scaled_train_data = ((train_data.T - mean)/std).T
        scaled_test_data = ((test_data.T - mean)/std).T
        mean = np.mean(scaled_train_data, axis = 0)
        var = np.var(scaled_train_data, axis = 0)

      for a in range(test_data.shape[0]):
        test_data_item = scaled_test_data[a]
        test_data_item = test_data_item[np.newaxis,:]
        #train_data_transposed = train_data

        total_data = np.concatenate((scaled_train_data, test_data_item), axis = 0)

        nn.fit(total_data)
        _ , neighbor = nn.kneighbors(test_data_item)


        neighbors[a][:] = neighbor[0][1:4]

      print(test_data.shape)
      umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'cosine') #initialize umap with desired hyperparams
      reduced_train_data = umap_reducer.fit_transform(scaled_train_data) #calculate umap and return reduced embeddings
      reduced_test_data = np.empty((test_data.shape[0], reduced_train_data.shape[1]))
      for j in range(test_data.shape[0]):
        #print(neighbors[j][0])
        #print(reduced_train_data[int(neighbors[j][0])])
        reduced_test_data[j] = np.mean( np.array([reduced_train_data[int(neighbors[j][0])], reduced_train_data[int(neighbors[j][1])], reduced_train_data[int(neighbors[j][2])]]), axis = 0)

      plt.figure()
      plt.title(pre_processing + '   test_set_size:' + str(test_size*100) +'%' + '   nr:' + str(i+1) )
      plt.scatter(reduced_train_data[:, 0], reduced_train_data[:, 1], s=0.1, color = 'silver')
      plt.scatter(reduced_test_data[:, 0], reduced_test_data[:, 1], s=0.1, color = 'crimson')
      plt.savefig(w_path + '/' + 'UMAP_cosine_afterpreprocessing' + '/' + pre_processing + 'test_set_size' + str(test_size*100) +'%' + '_nr' + str(i+1) + '.pdf', bbox_inches='tight')



In [ ]:
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


#loading the data
data = np.load(path)

if os.path.exists(w_path +'/' + 'UMAP_nn_after_preprocessing') == False:
  os.mkdir(w_path +'/' +  'UMAP_nn_after_preprocessing')

for i in range(5):
  for test_size in [0.3, 0.1, 0.05]:
    for pre_processing in ['feature_stand','norm_vecs']:

      train_data, test_data = train_test_split(data, test_size= test_size)
      nn = NearestNeighbors(n_neighbors=4)
      neighbors = np.empty((test_data.shape[0],3))


      #checking which means of data pre-processing to use
      if pre_processing == 'norm_vecs':
        scale_factors_train = np.sum(train_data, axis = 1)
        scale_factors_train = scale_factors_train[:, np.newaxis]
        scaled_train_data = train_data / scale_factors_train
        scale_factors_test = np.sum(test_data, axis = 1)
        scale_factors_test = scale_factors_test[:, np.newaxis]
        scaled_test_data = test_data / scale_factors_test
      else:
        #sc = StandardScaler()
        #sc.fit(train_data)
        #scaled_train_data = sc.transform(train_data)
        #scaled_test_data = sc.transform(test_data)
        #X_orig= sc.inverse_tranform(X_scaled)
        mean = np.mean(train_data, axis = 0)
        mean = mean[:, np.newaxis]
        #mean_train = np.broadcast_to(mean, (mean.shape[0], train_data.shape[0]))
        #print(mean_train.shape)
        #mean_test = np.broadcast_to(mean, (mean.shape[0], test_data.shape[0]))
        std = np.std(train_data, axis = 0)
        std = std[:, np.newaxis]
        scaled_train_data = ((train_data.T - mean)/std).T
        scaled_test_data = ((test_data.T - mean)/std).T
        mean = np.mean(scaled_train_data, axis = 0)
        var = np.var(scaled_train_data, axis = 0)

      for a in range(test_data.shape[0]):
        test_data_item = scaled_test_data[a]
        test_data_item = test_data_item[np.newaxis,:]
        #train_data_transposed = train_data

        total_data = np.concatenate((scaled_train_data, test_data_item), axis = 0)

        nn.fit(total_data)
        _ , neighbor = nn.kneighbors(test_data_item)


        neighbors[a][:] = neighbor[0][1:4]

      print(test_data.shape)
      umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'euclidean') #initialize umap with desired hyperparams
      reduced_train_data = umap_reducer.fit_transform(scaled_train_data) #calculate umap and return reduced embeddings
      reduced_test_data = np.empty((test_data.shape[0], reduced_train_data.shape[1]))
      for j in range(test_data.shape[0]):
        #print(neighbors[j][0])
        #print(reduced_train_data[int(neighbors[j][0])])
        reduced_test_data[j] = np.mean( np.array([reduced_train_data[int(neighbors[j][0])], reduced_train_data[int(neighbors[j][1])], reduced_train_data[int(neighbors[j][2])]]), axis = 0)

      plt.figure()
      plt.title(pre_processing + '   test_set_size:' + str(test_size*100) +'%' + '   nr:' + str(i+1) )
      plt.scatter(reduced_train_data[:, 0], reduced_train_data[:, 1], s=0.1, color = 'silver')
      plt.scatter(reduced_test_data[:, 0], reduced_test_data[:, 1], s=0.1, color = 'crimson')
      plt.savefig(w_path + '/' + 'UMAP_nn_after_preprocessing' + '/' + pre_processing + 'test_set_size' + str(test_size*100) +'%' + '_nr' + str(i+1) + '.pdf', bbox_inches='tight')



(619, 768)
(619, 768)
(207, 768)
(207, 768)
(104, 768)
(104, 768)
(619, 768)
(619, 768)
(207, 768)
(207, 768)
(104, 768)
(104, 768)
(619, 768)
(619, 768)
(207, 768)
(207, 768)
(104, 768)
(104, 768)
(619, 768)
(619, 768)
(207, 768)
(207, 768)
(104, 768)
(104, 768)
(619, 768)
(619, 768)
(207, 768)
(207, 768)
(104, 768)
(104, 768)


In [ ]:
#checking which data to import
if data_generation == 'adapters':
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_adapters.npy'
else:
  path='/content/gdrive/MyDrive/DIm_red_Clustering/user_embeddings_32d.npy'


#loading the data
data = np.load(path)

if os.path.exists(w_path +'/' + 'UMAP_nearestneighbor') == False:
  os.mkdir(w_path +'/' +  'UMAP_nearestneighbor')

for i in range(5):
  for test_size in [0.3, 0.1, 0.05]:
    for pre_processing in ['feature_stand','norm_vecs']:

      train_data, test_data = train_test_split(data, test_size= test_size)
      nn = NearestNeighbors(n_neighbors=2)
      neighbors = np.empty((test_data.shape[0],1))
      for a in range(test_data.shape[0]):
        test_data_item = test_data[a]
        test_data_item = test_data_item[np.newaxis,:]
        #train_data_transposed = train_data

        total_data = np.concatenate((train_data, test_data_item), axis = 0)

        nn.fit(total_data)
        _ , neighbor = nn.kneighbors(test_data_item)


        neighbors[a][:] = neighbor[0][1]




      #checking which means of data pre-processing to use
      if pre_processing == 'norm_vecs':
        scale_factors_train = np.sum(train_data, axis = 1)
        scale_factors_train = scale_factors_train[:, np.newaxis]
        scaled_train_data = train_data / scale_factors_train
        scale_factors_test = np.sum(test_data, axis = 1)
        scale_factors_test = scale_factors_test[:, np.newaxis]
        scaled_test_data = test_data / scale_factors_test
      else:
        #sc = StandardScaler()
        #sc.fit(train_data)
        #scaled_train_data = sc.transform(train_data)
        #scaled_test_data = sc.transform(test_data)
        #X_orig= sc.inverse_tranform(X_scaled)
        mean = np.mean(train_data, axis = 0)
        mean = mean[:, np.newaxis]
        #mean_train = np.broadcast_to(mean, (mean.shape[0], train_data.shape[0]))
        #print(mean_train.shape)
        #mean_test = np.broadcast_to(mean, (mean.shape[0], test_data.shape[0]))
        std = np.std(train_data, axis = 0)
        std = std[:, np.newaxis]
        scaled_train_data = ((train_data.T - mean)/std).T
        scaled_test_data = ((test_data.T - mean)/std).T
        mean = np.mean(scaled_train_data, axis = 0)
        var = np.var(scaled_train_data, axis = 0)


      umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.3, metric = 'euclidean') #initialize umap with desired hyperparams
      reduced_train_data = umap_reducer.fit_transform(scaled_train_data) #calculate umap and return reduced embeddings
      reduced_test_data = np.empty((test_data.shape[0], reduced_train_data.shape[1]))
      for j in range(test_data.shape[0]):
        #print(neighbors[j][0])
        #print(reduced_train_data[int(neighbors[j][0])])
        reduced_test_data[j] = reduced_train_data[int(neighbors[j])]

      plt.figure()
      plt.title(pre_processing + '   test_set_size:' + str(test_size*100) +'%' + '   nr:' + str(i+1) )
      plt.scatter(reduced_train_data[:, 0], reduced_train_data[:, 1], s=0.1, color = 'silver')
      plt.scatter(reduced_test_data[:, 0], reduced_test_data[:, 1], s=0.1, color = 'crimson')
      plt.savefig(w_path + '/' + 'UMAP_nearestneighbor' + '/' + pre_processing + 'test_set_size' + str(test_size*100) +'%' + '_nr' + str(i+1) + '.pdf', bbox_inches='tight')



##6.2 Sanity Check: Proximity Preservation
In this section, we want to see if the proximity of two points in the higher-dimensional embedding is preserved in the lower-dimensional embedding.

In order to do this, we will perform three checks:

**Check 1:** Does the single nearest neighbor of a point in the high-dimensional embedding lie within the same cluster as this point in the low-dimensional embedding?

**Check 2:** Do the three nearest neighbors of a point in the high-dimensional embedding lie within the same cluster as this point in the low-dimensional embedding?

**Check 3:** Does a point that is not in close proximity of another point in the high-dimensional embedding lie in a different cluster than this point in the low-dimensional embedding?




In [ ]:
#Prepare folders to store resulting plots in

sanity_check_path = w_path + 'UMAP_sanity_checks'
check_1_path = sanity_check_path + '/' + 'check1'
check_2_path = sanity_check_path + '/' + 'check2'
check_3_path = sanity_check_path + '/' + 'check3'

if os.path.exists(sanity_check_path) == False:
  os.mkdir(sanity_check_path)
if os.path.exists(check_1_path) == False:
  os.mkdir(check_1_path)
if os.path.exists(check_2_path) == False:
  os.mkdir(check_2_path)
if os.path.exists(check_3_path) == False:
  os.mkdir(check_3_path)

### 6.2.1 Check 1

In [ ]:
color_iter = ['crimson','olive','cyan','gold']
nr_embeddings = data.shape[0]
idx = np.random.randint(nr_embeddings, size=4)
neighbors = np.empty((4,2))

for pre_processing in ['none', 'feature_stand', 'norm_vecs']:
  if pre_processing == 'norm_vecs':
    scale_factors = np.sum(data, axis = 1)
    scale_factors = scale_factors[:, np.newaxis]
    processed_data = data / scale_factors
  elif pre_processing =='feature_stand':
    processed_data = StandardScaler().fit_transform(data)
  else:
    processed_data = data
  for measure in ['euclidean', 'cosine','manhattan']: #iteration over different reduced dim embeddings
    for i in range(4):
      sample_highdim = data[idx[i]]
      nn = NearestNeighbors(n_neighbors=2, metric = measure)
      nn.fit(data)
      sample_highdim = data[idx[i]]
      sample_highdim = sample_highdim[np.newaxis,:]
      _ , neighbor = nn.kneighbors(sample_highdim)
      neighbors[i] = neighbor
    neighbors = np.asarray(neighbors, dtype = 'int')
    #samples = data[idx,:]
    for n_neighbors in [2, 5,10,30]:
      for min_dist in [0.2, 0.4]:

        umap_reducer = umap.UMAP(n_components = 2, n_neighbors = n_neighbors, min_dist = min_dist, metric = measure) #initialize umap with desired hyperparams
        reduced_data = umap_reducer.fit_transform(processed_data) #calculate umap and return reduced embeddings
        plt.figure()
        plt.title(str(n_neighbors) +'   ' + str(min_dist) +'   ' + measure + '\n' + pre_processing)
        plt.scatter(reduced_data[:, 0], reduced_data[:, 1], s=0.1, color = 'silver')
        for i in range(4):
          #sample = reduced_data[idx[i]]
          #sample = sample[np.newaxis,:]
          neighbor_embeddings = reduced_data[neighbors[i],:]
          color = color_iter[i]
          plt.scatter(neighbor_embeddings[:, 0], neighbor_embeddings[:, 1], s=0.3, color = color)
        plt.savefig(check_1_path + '/' + pre_processing + '_' + measure + '_' + str(n_neighbors) +'_' + str(min_dist) + '.pdf', bbox_inches='tight')

### 6.2.2 Check 2

In [ ]:
# Trying for high n_neighbor hyperparameter

color_iter = ['crimson','olive','cyan','gold']
nr_embeddings = data.shape[0]
idx = np.random.randint(nr_embeddings, size=4)
neighbors = np.empty((4,20))

for pre_processing in ['none', 'feature_stand', 'norm_vecs']:
  if pre_processing == 'norm_vecs':
    scale_factors = np.sum(data, axis = 1)
    scale_factors = scale_factors[:, np.newaxis]
    processed_data = data / scale_factors
  elif pre_processing =='feature_stand':
    processed_data = StandardScaler().fit_transform(data)
  else:
    processed_data = data
  for measure in ['euclidean', 'cosine','manhattan']: #iteration over different reduced dim embeddings
    for i in range(4):
      sample_highdim = data[idx[i]]
      nn = NearestNeighbors(n_neighbors=20, metric = measure)
      nn.fit(data)
      sample_highdim = data[idx[i]]
      sample_highdim = sample_highdim[np.newaxis,:]
      _ , neighbor = nn.kneighbors(sample_highdim)
      neighbors[i] = neighbor
    neighbors = np.asarray(neighbors, dtype = 'int')
    #samples = data[idx,:]
    for n_neighbors in [2, 5,10,30]:
      for min_dist in [0.2, 0.4]:

        umap_reducer = umap.UMAP(n_components = 2, n_neighbors = n_neighbors, min_dist = min_dist, metric = measure) #initialize umap with desired hyperparams
        reduced_data = umap_reducer.fit_transform(processed_data) #calculate umap and return reduced embeddings
        plt.figure()
        plt.title(str(n_neighbors) +'   ' + str(min_dist) +'   ' + measure + '\n' + pre_processing)
        plt.scatter(reduced_data[:, 0], reduced_data[:, 1], s=0.1, color = 'silver')
        for i in range(4):
          #sample = reduced_data[idx[i]]
          #sample = sample[np.newaxis,:]
          neighbor_embeddings = reduced_data[neighbors[i],:]
          color = color_iter[i]
          plt.scatter(neighbor_embeddings[:, 0], neighbor_embeddings[:, 1], s=0.3, color = color)
        plt.savefig(check_2_path + '/' + '20points' + '_' + pre_processing + '_' + measure + '_' + str(n_neighbors) +'_' + str(min_dist) + '.pdf', bbox_inches='tight')

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ma

### 6.2.3 Check 3

In [ ]:
color_iter = ['crimson','olive','cyan','gold']
for pre_processing in ['none', 'feature_stand', 'norm_vecs']:

  if pre_processing == 'norm_vecs':
    scale_factors = np.sum(data, axis = 1)
    scale_factors = scale_factors[:, np.newaxis]
    processed_data = data / scale_factors
  elif pre_processing =='feature_stand':
    processed_data = StandardScaler().fit_transform(data)
  else:
    processed_data = data

  for metric in ['euclidean', 'cosine', 'manhattan']:
    n_neighbors = 2
    min_dist = 0.4
    umap_reducer = umap.UMAP(n_components = 2, n_neighbors = n_neighbors, min_dist = min_dist, metric = metric) #initialize umap with desired hyperparams
    reduced_data = umap_reducer.fit_transform(processed_data) #calculate umap and return reduced embeddings

    if metric == 'manhattan':
      metric_c = 'cityblock'
    else:
      metric_c = metric
    print(metric + metric_c)
    distance_vec = pdist(data, metric = metric_c)
    distance_matrix = squareform(distance_vec)
    for testpoints in range(4):
      max_distance, [i,j] = np.nanmax(distance_matrix), np.unravel_index( distance_matrix.argmax(), distance_matrix.shape )
      #print(np.linalg.norm(data[i] - data[j]) - max_distance)
      #print('manhattan' + str(distance.cityblock(data[i], data[j]) - max_distance))
      #print('cosine'+ str(distance.cosine(data[i], data[j]) - max_distance))
      #print(distance_matrix[i,j])
      #print(i)
      #print(j)
      distance_matrix[i,j] = 0
      distance_matrix[j,i] = 0
      color = color_iter[testpoints]
      plt.figure()
      plt.title('Distant Points' + '\n' + pre_processing + str(n_neighbors) +'   ' + str(min_dist) +'   ' + metric )
      plt.scatter(reduced_data[:, 0], reduced_data[:, 1], s=0.1, color = 'silver')
      plt.scatter(reduced_data[i, 0], reduced_data[i, 1], s=0.3, color = color)
      plt.scatter(reduced_data[j, 0], reduced_data[j, 1], s=0.3, color = color)
      print(np.linalg.norm(reduced_data[i] - reduced_data[j]) )
      print('manhattan' + str(distance.cityblock(reduced_data[i], reduced_data[j])))
      print('cosine'+ str(distance.cosine(reduced_data[i], reduced_data[j])))
      plt.savefig(check_3_path + '/' + pre_processing + '_' + metric + '_' + str(n_neighbors) +'_' + str(min_dist) + '.pdf', bbox_inches='tight')


euclideaneuclidean
28.872692
manhattan39.003998
cosine1.732623815536499
28.889385
manhattan39.022457
cosine1.7324561476707458
28.855011
manhattan39.031185
cosine1.7258954644203186
28.893576
manhattan39.037758
cosine1.7324785590171814
cosinecosine
19.590925
manhattan24.320436
cosine1.4616790413856506
19.85264
manhattan24.470734
cosine1.5047436952590942
19.502613
manhattan23.74619
cosine1.5149433612823486
20.14878
manhattan22.343721
cosine1.8144596815109253


/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


manhattancityblock
17.416327
manhattan22.863401
cosine1.9120588898658752
24.273214
manhattan32.5449
cosine1.1974610388278961
17.373507
manhattan22.83827
cosine1.9056857824325562
17.14573
manhattan22.471436
cosine1.9089009761810303
euclideaneuclidean
8.085731
manhattan10.304594
cosine0.12791383266448975
8.051851
manhattan10.244055
cosine0.12796783447265625
8.047795
manhattan10.2138
cosine0.12751871347427368
7.8789754
manhattan9.927461
cosine0.12224125862121582
cosinecosine
15.4954
manhattan20.543406
cosine0.9266278371214867
18.883387
manhattan22.175663
cosine1.1907411515712738
18.655613
manhattan21.624691
cosine1.2083291858434677
18.243114
manhattan20.253387
cosine1.2636833488941193


/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


manhattancityblock
27.096205
manhattan28.689184
cosine1.8188676238059998
21.473225
manhattan28.610256
cosine0.9060535132884979
27.113916
manhattan28.693752
cosine1.8183984756469727
27.11726
manhattan28.692442
cosine1.819487452507019
euclideaneuclidean
5.2437997
manhattan5.8041344
cosine0.4315829873085022
14.718917
manhattan18.966797
cosine1.0501969009637833
20.629534
manhattan25.012136
cosine1.8915109634399414
14.779425
manhattan18.305595
cosine0.8390147387981415
cosinecosine
0.14867732
manhattan0.20231593
cosine2.9325485229492188e-05
0.5890975
manhattan0.8185798
cosine0.0008529424667358398
0.65066296
manhattan0.82714266
cosine0.00025600194931030273
3.920839
manhattan5.2097654
cosine0.10112309455871582


/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:274: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


manhattancityblock
9.1292925
manhattan11.49642
cosine0.5921822786331177
22.860207
manhattan32.328575
cosine1.98752760887146
21.906595
manhattan30.585163
cosine1.9268772602081299
23.075054
manhattan23.465057
cosine1.7175220251083374


In [ ]:
n_neighbors = 2
min_dist = 0.4
umap_reducer = umap.UMAP(n_components = 2, n_neighbors = 2, min_dist = 0.4, metric = 'euclidean') #initialize umap with desired hyperparams
reduced_data = umap_reducer.fit_transform(processed_data) #calculate umap and return reduced embeddings
dbscan = DBSCAN(eps= 3/2, min_samples = 50)
dbscan.fit(reduced_data)
labels = dbscan.labels_
idx_c1 = [i for i in range(len(labels)) if labels[i] == 1]
c1_emb = [data[idx_c1[i]] for i in range(len(idx_c1))]
idx_c2 = [i for i in range(len(labels)) if labels[i] == 2]
c2_emb = [data[idx_c2[i]] for i in range(len(idx_c2))]
idx_c3 = [i for i in range(len(labels)) if labels[i] == 3]
c3_emb = [data[idx_c3[i]] for i in range(len(idx_c3))]

print(idx_c1[0] in idx_c2)
print(idx_c1[0] in idx_c3)
print(idx_c1[10] in idx_c2)
print(idx_c1[10] in idx_c3)
print(idx_c1[15] in idx_c2)
print(idx_c1[15] in idx_c3)

distance_vec = pdist(data)
distance_matrix = squareform(distance_vec)
max_distance, median, [i,j] = np.nanmax(distance_matrix), np.median(distance_matrix),np.unravel_index( distance_matrix.argmax(), distance_matrix.shape )
print(max_distance)
print(median)

neighbors = np.empty((1,2))
idx = np.random.randint(data.shape[0])
sample_highdim = data[idx]
nn = NearestNeighbors(n_neighbors=2, metric = 'euclidean')
nn.fit(data)
sample_highdim_a = sample_highdim[np.newaxis,:]
_ , neighbors_array = nn.kneighbors(sample_highdim_a)
print(neighbors_array.shape)
neighbor = neighbors_array[0][1]
min_distance = distance.euclidean(sample_highdim, data[neighbor])
print(min_distance)

distance_vec = pdist(c1_emb)
distance_matrix = squareform(distance_vec)
c1_max_distance, c1_median, [c1_i,c1_j] = np.nanmax(distance_matrix), np.median(distance_matrix),np.unravel_index( distance_matrix.argmax(), distance_matrix.shape )
print('\n')
print(c1_max_distance)
print(c1_median)

distance_vec = pdist(c2_emb)
distance_matrix = squareform(distance_vec)
c2_max_distance, c2_median, [c2_i,c2_j] = np.nanmax(distance_matrix), np.median(distance_matrix), np.unravel_index( distance_matrix.argmax(), distance_matrix.shape )
print('\n')
print(c2_max_distance)
print(c2_median)

distance_vec = pdist(c3_emb)
distance_matrix = squareform(distance_vec)
c3_max_distance, c3_median, [c3_i,c3_j] = np.nanmax(distance_matrix),np.median(distance_matrix), np.unravel_index( distance_matrix.argmax(), distance_matrix.shape )
print('\n')
print(c3_max_distance)
print(c3_median)


False
False
False
False
False
False
29.953537422259267
6.72263167680873
(1, 2)
4.5948486328125


22.94293859250076
6.904077559438377


17.136880389018163
6.333739329654997


13.245203463214919
5.725724270397775


In [ ]:
neighbors = np.empty((1,10))
idx = np.random.randint(data.shape[0])
sample_highdim = data[idx]
nn = NearestNeighbors(n_neighbors=10, metric = 'euclidean')
nn.fit(data)
sample_highdim_a = sample_highdim[np.newaxis,:]
_ , neighbors_array = nn.kneighbors(sample_highdim_a)
neighbor = neighbors_array[0][1:9]
min_distance = distance.euclidean(sample_highdim, data[neighbor[0]])
print(min_distance)
min_distance = distance.euclidean(sample_highdim, data[neighbor[1]])
print(min_distance)
min_distance = distance.euclidean(sample_highdim, data[neighbor[2]])
print(min_distance)
min_distance = distance.euclidean(sample_highdim, data[neighbor[3]])
print(min_distance)
min_distance = distance.euclidean(sample_highdim, data[neighbor[4]])
print(min_distance)
min_distance = distance.euclidean(sample_highdim, data[neighbor[5]])
print(min_distance)
min_distance = distance.euclidean(sample_highdim, data[neighbor[6]])
print(min_distance)
min_distance = distance.euclidean(sample_highdim, data[neighbor[7]])
print(min_distance)




4.248859405517578
4.33917760848999
4.3432722091674805
4.383040428161621
4.47647762298584
4.531210899353027
4.532975673675537
4.560563564300537


# 7. Clustering
Finally, we will have to cluster the reduced dimensionality embeddings.

In this section, we will try out different clustering algorithms with different hyperparameters each. We will furthermore use the xxx metric to support qualitative assesments of the clustering, enabling us to choose our final set of hyperparameters.

Please note that this metric calculates the quality score of the clustering based on distance, i.e. factors such as cluster diameter, average distance between cluster points, distance between separate clusters etc..
Yet, as UMAP focuses on preserving the local structure of the data, the distances between clusters as well as the size of the clusters themselves are not interpretable. Moreover, some of the used clustering algorithms do not cluster according to distance but according to density, distribution, or graph structures. Therefore, the used metric does not perfectly evaluate the quality of the clustering and merely serves as an approximative assistance for the evalutation.

## 7.1 K-Means

In [ ]:
#performing k-means clustering by iterating over plausible hyperparameter values

#creating dict to track ch scores
ch_tracker = {}
print(iter_array)
#iterating over the different hyperparameter values and reduced dimensionality embeddings
for data_generation, pre_processing, dim_reduction, n_dims, n_neighbors, min_dist, measure in iter_array: #iteration over different reduced dim embeddings
  reduced_data = embeddings[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure]
  print(reduced_data)
  for n_clusters in np.concatenate((range(2,10,1),range(10,101,5))): #iterating over number of clusters

    kmeans = KMeans(init="random", n_clusters=n_clusters, n_init=10, max_iter=300, random_state=42)
    kmeans.fit(reduced_data)
    labels = kmeans.labels_ #extracting labels of each sample
    ch_score = round(metrics.calinski_harabasz_score(reduced_data, labels),2)
    ch_tracker[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure + str(n_clusters)] = ch_score

    if n_dims == '2':
      #configuring plot settings, one color for each created label
      plt.figure()
      plt.scatter(reduced_data[:,0], reduced_data[:,1], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score) + '    nr clusters:' + str(n_clusters) + '   Kmeans', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'k_means'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(n_clusters) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')


    else:
      plt.figure()
      ax = plt.axes(projection='3d')
      ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score)+ '    nr clusters:' + str(n_clusters) + '   Kmeans', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'k_means'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(n_clusters) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')




[['adapters' 'feature_stand' 'UMAP' '2' '2' '0.4' 'cosine']
 ['adapters' 'feature_stand' 'UMAP' '2' '2' '0.3' 'cosine']
 ['adapters' 'feature_stand' 'UMAP' '2' '2' '0.6' 'cosine']
 ['adapters' 'feature_stand' 'UMAP' '2' '2' '0.3' 'euclidean']
 ['adapters' 'feature_stand' 'UMAP' '2' '2' '0.7' 'euclidean']
 ['adapters' 'feature_stand' 'UMAP' '2' '2' '0.9' 'euclidean']
 ['adapters' 'feature_stand' 'UMAP' '2' '2' '0.3' 'euclidean']
 ['adapters' 'feature_stand' 'UMAP' '2' '2' '0.7' 'manhattan']
 ['adapters' 'feature_stand' 'UMAP' '2' '2' '0.5' 'euclidean']
 ['adapters' 'feature_stand' 'UMAP' '2' '2' '0.4' 'euclidean']
 ['adapters' 'feature_stand' 'UMAP' '3' '2' '0.4' 'cosine']
 ['adapters' 'feature_stand' 'UMAP' '3' '2' '0.3' 'cosine']
 ['adapters' 'feature_stand' 'UMAP' '3' '2' '0.6' 'cosine']
 ['adapters' 'feature_stand' 'UMAP' '3' '2' '0.5' 'cosine']
 ['adapters' 'feature_stand' 'UMAP' '3' '2' '0.6' 'euclidean']
 ['adapters' 'feature_stand' 'UMAP' '3' '2' '0.5' 'manhattan']
 ['adapters' 

<ipython-input-19-6a3785b19869>:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


KeyboardInterrupt: ignored

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


[[-1.4098315 -1.2458988]
 [-0.19835   15.155466 ]
 [ 5.0484977 15.255997 ]
 ...
 [17.869959   1.8534476]
 [16.638546  10.022413 ]
 [ 5.547081  14.886041 ]]
[[-1.9019655 -1.3828804]
 [ 0.7018281 13.06356  ]
 [ 4.6110716 16.978762 ]
 ...
 [15.668492  -2.1295192]
 [16.923805   9.526438 ]
 [ 5.3650784 16.781603 ]]
[[ 5.214946    0.12763412]
 [22.352123    6.529311  ]
 [19.70649     5.9890723 ]
 ...
 [ 6.7625623   1.7476624 ]
 [-1.931388   13.321524  ]
 [ 3.0041745  13.011845  ]]
[[ 9.590451    0.4823868 ]
 [16.041098   10.058055  ]
 [18.187893    8.43209   ]
 ...
 [ 8.312008    0.6020971 ]
 [-0.23025209 15.540795  ]
 [ 2.9037366  12.9291725 ]]


...

[[10.035096    0.6314846 ]
 [17.263569   12.44225   ]
 [16.062506   11.140568  ]
 ...
 [ 8.945465   -0.33891827]
 [ 0.51793516 15.834822  ]
 [ 3.3785067  17.647436  ]]
[[ 5.214946    0.12763412]
 [22.352123    6.529311  ]
 [19.70649     5.9890723 ]
 ...
 [ 6.7625623   1.7476624 ]
 [-1.931388   13.321524  ]
 [ 3.0041745  13.011845  ]]


KeyboardInterrupt: ignored

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


[[ 1.9714965 10.871984 ]
 [ 6.218172   2.109453 ]
 [-8.533468  -2.4101446]
 ...
 [ 2.9432032 10.303691 ]
 [13.724179  -3.5490363]
 [-6.268562  -0.5372991]]
[[ 9.600774   5.5791345]
 [21.47772    2.8897264]
 [18.237797   4.523575 ]
 ...
 [ 9.93182    4.086981 ]
 [ 0.5093076 11.334043 ]
 [ 2.5185466 14.410665 ]]


KeyboardInterrupt: ignored

In [ ]:
#Determining the best clusters according to ch score and printing them in table
largest_keys = sorted(ch_tracker, key=ch_tracker.get, reverse=True)[:5]
largest_vals = [ch_tracker[x] for x in largest_keys]
length = len(largest_vals)
heading = np.empty(length, dtype = str)
heading[:] = 'value'
pd.DataFrame(largest_vals, index = largest_keys, columns=["values"])

,values


## 7.2 Agglomerative Clustering


In [ ]:
#performing Agglomerative clustering by iterating over plausible hyperparameter values

#creating dict to track ch scores
ch_tracker = {}

#iterating over the different hyperparameter values and reduced dimensionality embeddings
for data_generation, pre_processing, dim_reduction, n_dims, n_neighbors, min_dist, measure in iter_array: #iteration over different reduced dim embeddings
  reduced_data = embeddings[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure]
  for n_clusters in np.concatenate((range(2,10,1),range(10,101,5))): #iterating over number of clusters
    agglo = AgglomerativeClustering(n_clusters = n_clusters)
    agglo.fit(reduced_data)
    labels =  agglo.labels_
    ch_score = round(metrics.calinski_harabasz_score(reduced_data, labels),2)
    ch_tracker[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure + str(n_clusters)] = ch_score

    if n_dims == '2':
      #configuring plot settings, one color for each created label
      plt.figure()
      plt.scatter(reduced_data[:,0], reduced_data[:,1], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score) + '    nr clusters:' + str(n_clusters) + '\n Agglo', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'agglo'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(n_clusters) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')


    else:
      plt.figure()
      ax = plt.axes(projection='3d')
      ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score)+ '    nr clusters:' + str(n_clusters)+ '\n Agglo', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'agglo'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(n_clusters) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')

In [ ]:
#Determining the best clusters according to ch score and printing them in table
largest_keys = sorted(ch_tracker, key=ch_tracker.get, reverse=True)[:5]
largest_vals = [ch_tracker[x] for x in largest_keys]
length = len(largest_vals)
heading = np.empty(length, dtype = str)
heading[:] = 'value'
pd.DataFrame(largest_vals, index = largest_keys, columns=["values"])

,values
adaptersfeature_standUMAP220.3euclidean100,23395.33
adaptersfeature_standUMAP220.3euclidean95,22551.37
adaptersfeature_standUMAP220.3euclidean90,21770.59
adaptersfeature_standUMAP220.3euclidean85,21021.86
adaptersfeature_standUMAP220.3euclidean80,20280.69


## 7.3 Spectral Clustering

In [ ]:
#performing Spectral clustering by iterating over plausible hyperparameter values


#creating dict to track ch scores
ch_tracker = {}

#iterating over the different hyperparameter values and reduced dimensionality embeddings
for data_generation, pre_processing, dim_reduction, n_dims, n_neighbors, min_dist, measure in iter_array: #iteration over different reduced dim embeddings
  reduced_data = embeddings[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure]
  for n_clusters in np.concatenate((range(2,10,1),range(10,101,5))): #iterating over different number of clusters
    spectral = SpectralClustering(n_clusters = n_clusters)
    spectral.fit(reduced_data)
    labels =  spectral.labels_
    ch_score = round(metrics.calinski_harabasz_score(reduced_data, labels),2)
    ch_tracker[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure + str(n_clusters)] = ch_score

    if n_dims == '2':
      #configuring plot settings, one color for each created label
      plt.figure()
      plt.scatter(reduced_data[:,0], reduced_data[:,1], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score) + '    nr clusters:' + str(n_clusters)+ '\n Spectral', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'spectral'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(n_clusters) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')


    else:
      plt.figure()
      ax = plt.axes(projection='3d')
      ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score)+ '    nr clusters:' + str(n_clusters)+ '\n Spectral', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'spectral'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(n_clusters) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')

In [ ]:
#Determining the best clusters according to ch score and printing them in table
largest_keys = sorted(ch_tracker, key=ch_tracker.get, reverse=True)[:5]
largest_vals = [ch_tracker[x] for x in largest_keys]
length = len(largest_vals)
heading = np.empty(length, dtype = str)
heading[:] = 'value'
pd.DataFrame(largest_vals, index = largest_keys, columns=["values"])

## 7.4 BIRCH Clustering

In [ ]:
#performing BIRCH clustering by iterating over plausible hyperparameter values

#creating dict to track ch scores
ch_tracker = {}

#iterating over the different hyperparameter values and reduced dimensionality embeddings
for data_generation, pre_processing, dim_reduction, n_dims, n_neighbors, min_dist, measure in iter_array: #iteration over different reduced dim embeddings
  reduced_data = embeddings[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure]
  for n_clusters in np.concatenate((range(2,10,1),range(10,101,5))): #iterating over number of clusters
    birch = Birch(n_clusters = n_clusters)
    birch.fit(reduced_data)
    labels =  birch.labels_
    ch_score = round(metrics.calinski_harabasz_score(reduced_data, labels))
    ch_tracker[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure + str(n_clusters)] = ch_score

    if n_dims == '2':
      #configuring plot settings, one color for each created label
      plt.figure()
      plt.scatter(reduced_data[:,0], reduced_data[:,1], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score) + '    nr clusters:' + str(n_clusters) + '\n BIRCH', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'BIRCH'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(n_clusters) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')


    else:
      plt.figure()
      ax = plt.axes(projection='3d')
      ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score)+ '    nr clusters:' + str(n_clusters) + '\n Birch', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'BIRCH'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(n_clusters) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')

<ipython-input-10-6b8dee831ef4>:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:726: ConvergenceWarning: Number of subclusters found (82) by BIRCH is less than (85). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:726: ConvergenceWarning: Number of subclusters found (82) by BIRCH is less than (90). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_birch.py:726: ConvergenceWarning: Number of subclusters found (82) by BIRCH is less than (95). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_

In [ ]:
#Determining the best clusters according to ch score and printing them in table
largest_keys = sorted(ch_tracker, key=ch_tracker.get, reverse=True)[:5]
largest_vals = [ch_tracker[x] for x in largest_keys]
length = len(largest_vals)
heading = np.empty(length, dtype = str)
heading[:] = 'value'
pd.DataFrame(largest_vals, index = largest_keys, columns=["values"])

,values
adaptersfeature_standUMAP220.3euclidean75,17358
adaptersfeature_standUMAP220.3euclidean80,16966
adaptersfeature_standUMAP220.3euclidean70,16880
adaptersfeature_standUMAP220.3euclidean85,16532
adaptersfeature_standUMAP220.3euclidean90,16532


## 7.5 Optics Clustering

In [ ]:
#performing OPTICS clustering by iterating over plausible hyperparameter values

#creating dict to track ch scores
ch_tracker = {}

#iterating over the different hyperparameter values and reduced dimensionality embeddings
for data_generation, pre_processing, dim_reduction, n_dims, n_neighbors, min_dist, measure in iter_array: #iteration over different reduced dim embeddings
  reduced_data = embeddings[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure]
  for min_samples in range(20,100,5): #iterating over number of samples in a neighborhood for a point to be considered as a core point
    optics = OPTICS(min_samples = min_samples)
    optics.fit(reduced_data)
    labels = optics.labels_
    ch_score = round(metrics.calinski_harabasz_score(reduced_data, labels))
    ch_tracker[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure + str(n_clusters)] = ch_score

    if n_dims == '2':
      #configuring plot settings, one color for each created label
      plt.figure()
      plt.scatter(reduced_data[:,0], reduced_data[:,1], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '   -  ' + pre_processing + '  -   ' + dim_reduction + ' \n ' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + ' \n ch score:'+ str(ch_score) + '    min samples:' + str(min_samples) +  ' \n Optics', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'Optics'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(min_samples) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')


    else:
      plt.figure()
      ax = plt.axes(projection='3d')
      ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + ' \n ch score:' + str(ch_score)+ '    min samples:' + str(min_samples) + '\n Optics', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'Optics'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(min_samples) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')

In [ ]:
#Determining the best clusters according to ch score and printing them in table
largest_keys = sorted(ch_tracker, key=ch_tracker.get, reverse=True)[:5]
largest_vals = [ch_tracker[x] for x in largest_keys]
length = len(largest_vals)
heading = np.empty(length, dtype = str)
heading[:] = 'value'
pd.DataFrame(largest_vals, index = largest_keys, columns=["values"])

,values
adaptersfeature_standUMAP220.7euclidean100,1739
adaptersnorm_vecsUMAP320.5manhattan100,1040
adaptersfeature_standUMAP320.6euclidean100,1027
adaptersfeature_standUMAP220.9euclidean100,992
adaptersfeature_standUMAP320.6manhattan100,756


## 7.6 DBScan Clustering

In [ ]:
#performing DBScan clustering by iterating over plausible hyperparameter values

#creating dict to track ch scores
ch_tracker = {}

#iterating over the different hyperparameter values and reduced dimensionality embeddings
for data_generation, pre_processing, dim_reduction, n_dims, n_neighbors, min_dist, measure in iter_array: #iteration over different reduced dim embeddings
  reduced_data = embeddings[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure]
  for min_samples in range(20,100,10): #iterating over number of samples in a neighborhood for a point to be considered as a core point
    for eps in range(1, 20, 1):
      dbscan = DBSCAN(eps= eps/2, min_samples = min_samples)
      dbscan.fit(reduced_data)
      labels = dbscan.labels_
    ch_score = round(metrics.calinski_harabasz_score(reduced_data, labels),2)
    ch_tracker[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure + str(n_clusters)] = ch_score

    if n_dims == '2':
      #configuring plot settings, one color for each created label
      plt.figure()
      plt.scatter(reduced_data[:,0], reduced_data[:,1], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score) + '    min samples:' + str(min_samples) + '    eps:'+ str(eps) + '\n DBScan', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'DBScan'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(min_samples) + str(eps) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')


    else:
      plt.figure()
      ax = plt.axes(projection='3d')
      ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score)+ '    min samples:' + str(min_samples)+ '    eps:'+ str(eps) +'\n DBScan', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'DBScan'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(min_samples) + str(eps) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')

In [ ]:
#Determining the best clusters according to ch score and printing them in table
largest_keys = sorted(ch_tracker, key=ch_tracker.get, reverse=True)[:5]
largest_vals = [ch_tracker[x] for x in largest_keys]
length = len(largest_vals)
heading = np.empty(length, dtype = str)
heading[:] = 'value'
pd.DataFrame(largest_vals, index = largest_keys, columns=["values"])

## 7.6 Gaussian Mixture Models

In [ ]:
#performing GMM clustering by iterating over plausible hyperparameter values

#creating dict to track ch scores
ch_tracker = {}

#iterating over the different hyperparameter values and reduced dimensionality embeddings
for data_generation, pre_processing, dim_reduction, n_dims, n_neighbors, min_dist, measure in iter_array: #iteration over different reduced dim embeddings
  reduced_data = embeddings[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure]
  for nr_components in np.concatenate((range(2,10,1),range(10,101,5))): #iterating over number of samples in a neighborhood for a point to be considered as a core point
      gmm = mixture.GMM(nr_components=nr_components)
      gmm.fit(reduced_data)
      labels = gmm.labels_
    ch_score = round(metrics.calinski_harabasz_score(reduced_data, labels),2)
    ch_tracker[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure + str(n_clusters)] = ch_score

    if n_dims == '2':
      #configuring plot settings, one color for each created label
      plt.figure()
      plt.scatter(reduced_data[:,0], reduced_data[:,1], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score) + '    min samples:' + str(min_samples) + '\n DBScan', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'gmm'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(nr_components) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')


    else:
      plt.figure()
      ax = plt.axes(projection='3d')
      ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score)+ '    min samples:' + str(min_samples)+ '\n DBScan', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'gmm'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(nr_components) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')

In [ ]:
#Determining the best clusters according to ch score and printing them in table
largest_keys = sorted(ch_tracker, key=ch_tracker.get, reverse=True)[:5]
largest_vals = [ch_tracker[x] for x in largest_keys]
length = len(largest_vals)
heading = np.empty(length, dtype = str)
heading[:] = 'value'
pd.DataFrame(largest_vals, index = largest_keys, columns=["values"])

##7.7 Bayesian Gaussian Mixture Model

In [ ]:
#performing GMM clustering by iterating over plausible hyperparameter values

#creating dict to track ch scores
ch_tracker = {}

#iterating over the different hyperparameter values and reduced dimensionality embeddings
for data_generation, pre_processing, dim_reduction, n_dims, n_neighbors, min_dist, measure in iter_array: #iteration over different reduced dim embeddings
  reduced_data = embeddings[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure]
  for nr_components in np.concatenate((range(2,10,1),range(10,101,5))): #iterating over number of samples in a neighborhood for a point to be considered as a core point
      vgmm = BayesianGaussianMixture(n_components=nr_components, random_state=42)
      vgmm.fit(reduced_data)
      labels = vgmm.labels_
    ch_score = round(metrics.calinski_harabasz_score(reduced_data, labels),2)
    ch_tracker[data_generation + pre_processing + dim_reduction + n_dims + n_neighbors+ min_dist + measure + str(n_clusters)] = ch_score

    if n_dims == '2':
      #configuring plot settings, one color for each created label
      plt.figure()
      plt.scatter(reduced_data[:,0], reduced_data[:,1], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score) + '    min samples:' + str(min_samples) + '\n DBScan', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'vgmm'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(nr_components) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')


    else:
      plt.figure()
      ax = plt.axes(projection='3d')
      ax.scatter3D(reduced_data[:, 0], reduced_data[:, 1], reduced_data[:, 2], c = np.take(colors, labels), s=0.1)
      plt.title(data_generation + '     ' + pre_processing + '     ' + dim_reduction + '\n' + 'n_neighbors:' + n_neighbors+ '     min_dist:' + min_dist + '     metric:' + measure + '\n' + 'ch score:'+str(ch_score)+ '    min samples:' + str(min_samples)+ '\n DBScan', fontsize=8)
      clustering_path = w_path + '/' + data_generation + '/' + pre_processing + '/' + dim_reduction + '/' + n_dims + '/' + 'vgmm'
      if os.path.exists(clustering_path) == False:
        os.mkdir(clustering_path)
      clustering_path = clustering_path + '/' + n_neighbors + '_' + min_dist + '_' + measure + str(nr_components) +'.pdf'
      plt.savefig(clustering_path , bbox_inches='tight')

In [ ]:
#Determining the best clusters according to ch score and printing them in table
largest_keys = sorted(ch_tracker, key=ch_tracker.get, reverse=True)[:5]
largest_vals = [ch_tracker[x] for x in largest_keys]
length = len(largest_vals)
heading = np.empty(length, dtype = str)
heading[:] = 'value'
pd.DataFrame(largest_vals, index = largest_keys, columns=["values"])

# 8. Testing Cluster Validity
In this section, we will validate our clustering.

We will proceed by checking the interpretability of the different clusters. In order to do so, we will sample user embeddings from the different resulting clusters. In a next step, we will transform them back to their original dimensionality to be able to pass them through our recommender system. The recommender system will then yield the representative word clouds for each cluster.

If the word clouds from different clusters vary significantly from one another and provide a basis for proper interpretability, the clustering will be deemed valid.

In [ ]:
#1. Perform UMAP with chosen hyperparams
#2. Perform Clustering with chosen hyperparams
#3. Sample 5 users from each cluster
#4. Backproject samples to original embedding dimensionality
#5. Send embeddings through recommender systems in cluster-batches to withdraw key words for each cluster
#6. Build word cloud from key-words